In [56]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request as req
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options

%pip install openpyxl

import openpyxl
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

book_cate = {'소설':'01','시/에세이':'03','인문':'05','경제/경영':'13','자기계발':'15','유아(0~7세)':'41'}
# 01 : 소설 , 03 : 시/에세이 , 05 : 인문 , 13 : 경제/경영 , 15 : 자기계발
url = 'https://product.kyobobook.co.kr/category/KOR/'+book_cate['유아(0~7세)']+'#?page=1&type=best&per=20'

User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
chrome_options = Options()
chrome_options.add_argument(f"user-agent={User_Agent }")

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

book_titles = list()
book_stories = list()
book_keyPicks = list()
keywordPick = list()
book_page_urls = list()

for i in range(1,3):
    page_links = driver.find_elements(By.CSS_SELECTOR,'#homeTabBest > div.switch_prod_wrap.view_type_list > ol > li > div.prod_area.horizontal > div.prod_thumb_box > a')

    for page_link in page_links:
        link = page_link.get_attribute('href')
        book_page_urls.append(link)
    if i== 50:
       break
    next_page = driver.find_element(By.CSS_SELECTOR,'#bestBottomPagi > button.btn_page.next')
    next_page.send_keys(Keys.ENTER)
    time.sleep(3)
driver.quit()


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [57]:
driver = webdriver.Chrome(options=chrome_options)
for url in book_page_urls:
    try:
        driver.get(url)
        time.sleep(3)
        title = driver.find_element(By.CSS_SELECTOR,'#contents > div.prod_detail_header > div > div.prod_detail_title_wrap > div > div.prod_title_box.auto_overflow_wrap > div.auto_overflow_contents > div > h1 > span')
        book_titles.append(title.text)
    except:
        book_titles.append('나중에 채우기!')

    try:
        # 키워드 Pick 수집
        keywordPicks = driver.find_elements(By.CSS_SELECTOR,'div.tab_list_wrap > ul >li> a > span.tab_text')
        keywordPick = list()
        for keyword in keywordPicks:
            keywordPick.append(keyword.text)
        book_keyPicks.append(keywordPick)
    except:
        book_keyPicks.append([])

    try:
         # 책 소개 수집
        story = driver.find_element(By.CSS_SELECTOR,'div.intro_bottom')
        book_stories.append(story.text)
    except:
        book_stories.append('')
        
driver.quit()

In [58]:
len(book_page_urls)

40

In [59]:
print(len(book_titles))
print(len(book_stories))
print(len(book_keyPicks))

40
40
40


In [60]:
book_df = pd.DataFrame([book_titles,book_stories,book_keyPicks],index=['상품명','책소개','키워드']).T
def delete_null(keywordList):
    keyword_list = list()
    for x in range(len(keywordList)):
        if keywordList[x] != '':
            keyword_list.append(keywordList[x])
    return keyword_list
book_df['키워드'] = book_df['키워드'].apply(delete_null)

book_category = ['소설','시/에세이','인문','경제/경영','자기계발']
file_name = book_category[2] + '키워드 Pick.xlsx'
book_df.to_excel(file_name)

In [61]:
book_df

,상품명,책소개,키워드
0,천둥 꼬마 선녀 번개 꼬마 선녀,한국 최초 노벨문학상 수상!\n소설가 한강이 어린이를 위해 쓴 단 한 권의 창작그림...,"[창작 그림책, 할머니, 하늘빛, 은빛, 상자, 단발머리]"
1,감정 호텔,"날마다 다른 감정들이 머물다 가는 곳,\n감정 호텔에 오신 것을 환영합니다!\n목소...","[볼로냐 국제 아동, 지배인, 손님, 세심, 환영, 마법]"
2,산타 할아버지의 첫 크리스마스,그림책계의 젊은 거장\n맥 바넷 × 시드니 스미스가 함께 그려 낸\n마법 같은 크리...,[]
3,페파의 서울 여행,가족과 함께 서울에 온 페파의 신나는 여행 이야기를 만나 보세요!\n와! 페파가 보...,"[신나, 영어 그림책, 인기 애니메이션, 경복궁, 궁궐, 케이블카]"
4,나뭇잎을 찾으면,"2022 미국 아마존 최고의 아동 도서 선정!\n계절의 변화를 자연스럽게 느끼며,\...","[초등 교과 연계, 누리 과정 연계, 쿠로, 면지, 가을, 돛단배]"
5,알사탕,"마음의 소리를 찾아서\n간절함이 부른 마법의 알사탕\n#새봄, 새로운 맛 : 202...","[볼로냐 국제 아동, 내면, 구슬, 반려, 문방구, 마법]"
6,동물 농장 사운드북,"와글와글 동물 농장에는 누가 살고 있을까요?\n청각과 언어 감각, 시각과 촉각을 기...","[아기, 놀이, 구멍, 촉감, 트랙터, 발달]"
7,기쁜 눈물,"“아가, 너무 기쁘면 나도 모르게 눈물이 난대”\n\n어떻게 된 일일까요?\n제가 ...","[위로, 불안, 감정, 응원, 그림책, 뱃속]"
8,클래식 뮤지컬,아름다운 음악과 함께 즐기는 뮤지컬 〈호두까기 인형〉\n발레곡 ‘호두까기 인형’ 다...,"[클라라, 과자, 병정, 겨울날, 크리스마스, 생쥐]"
9,캐치티니핑 캐릭터도감,"- 시즌1부터 시즌4까지 100마리가 넘는 역대 티니핑 총출동, 캐치티니핑 도감 출...","[로열, 티니, 컬러, 프린세스, 스티커, 앨범]"
